In [2]:
pip install selenium

  Using cached selenium-4.5.0-py3-none-any.whl (995 kB)
  Using cached trio_websocket-0.9.2-py3-none-any.whl (16 kB)
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached exceptiongroup-1.0.0-py3-none-any.whl (12 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install bs4

  Using cached bs4-0.0.1-py3-none-any.whl
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached soupsieve-2.3.2.post1-py3-none-any.whl (37 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install webdriver_manager

Note: you may need to restart the kernel to use updated packages.


# <center> Kayak Scraper </center>

In [28]:
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm

In [8]:
# path to chromedriver
chromedriver_path =  "C:/Users/Prince Raghuvanshi/Downloads/chromedriver_win32/chromedriver.exe"

In [29]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

options = webdriver.ChromeOptions()
options.binary_location = r"C:/Program Files/Google/Chrome/Application/chrome.exe"
chrome_driver_binary = r"C:/Program Files/Google/Chrome/Application/chromedriver.exe"
driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
#driver.get('http://www.google.com')

C:\Users\PRINCE~1\AppData\Local\Temp/ipykernel_26216/1579610596.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
C:\Users\PRINCE~1\AppData\Local\Temp/ipykernel_26216/1579610596.py:7: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


In [ ]:
# launching the driver
driver = webdriver.Chrome(chrome_driver_binary)



### User Input

In [30]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")


Please enter -1 when done.
----------


From which city?
 NYC
Where to?
 BOS


----------


From which city?
 IAD
Where to?
 LAX


----------


From which city?
 -1



Routes:
NYC => BOS
IAD => LAX


In [31]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only  2022-12-02
End Date, Please use YYYY-MM-DD format only  2022-12-05


### Define functions for scraping

In [32]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list


## Scraping

In [ ]:
for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination","Duration" ,"Total stops", "Price","Date"]
    df = pd.DataFrame(columns = column_names)
    for j in tqdm(range(num_days+1)):
        
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            #driver = webdriver.Chrome(chrome_driver_binary)#, chrome_options=chromeOptions)
            driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
            
        url = f"https://www.en.kayak.sa/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        #url = f"https://www.priceline.com/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)
        
        # click show more button to get all flights
        try:
            show_more_button = driver.find_element_by_xpath('//a[@class = "moreButton"]')
        except:
            
            # in case a captcha appears, require input from user so that the for loop pauses and the user can continue the
            # loop after solving the captcha
            input("Please solve the captcha then enter anything here to resume scraping.")
            
        while True:
            try:
                show_more_button.click()
                driver.implicitly_wait(10)
            except:
                break
    
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        df = df.append(pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops' : total_stops,
            'Price' : prices,
            'Date' : start_date+j
                                    }))
        
    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n','', regex=True)
    df = df.reset_index(drop = True)
    
    # save data as csv file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv',index=False)
    print(f"Succesfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv ")
    
driver.quit()

  0%|          | 0/4 [00:00<?, ?it/s]C:\Users\PRINCE~1\AppData\Local\Temp/ipykernel_26216/4000765959.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)
C:\Users\PRINCE~1\AppData\Local\Temp/ipykernel_26216/4000765959.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_driver_binary, chrome_options=options)


Please solve the captcha then enter anything here to resume scraping. g


 25%|██▌       | 1/4 [00:25<01:16, 25.40s/it]

Please solve the captcha then enter anything here to resume scraping. f


 50%|█████     | 2/4 [00:44<00:43, 21.84s/it]

...

 25%|██▌       | 1/4 [00:34<01:44, 34.68s/it]

Please solve the captcha then enter anything here to resume scraping. f


 50%|█████     | 2/4 [02:36<02:52, 86.10s/it]

Please solve the captcha then enter anything here to resume scraping. g


 75%|███████▌  | 3/4 [02:59<00:57, 57.18s/it]